In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch import nn
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from text_recognizer.networks.unet import UNet

In [3]:
net = UNet()

In [4]:
x = torch.rand(1, 1, 256, 256)

In [5]:
net.encoder_blocks

ModuleList(
  (0): _DilationBlock(
    (activation): ELU(alpha=1.0, inplace=True)
    (conv): Sequential(
      (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(6, 6), dilation=(3, 3))
      (1): ELU(alpha=1.0, inplace=True)
    )
    (conv1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
      (1): ELU(alpha=1.0, inplace=True)
    )
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (down_sampling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): _DilationBlock(
    (activation): ELU(alpha=1.0, inplace=True)
    (conv): Sequential(
      (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(6, 6), dilation=(3, 3))
      (1): ELU(alpha=1.0, inplace=True)
    )
    (conv1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ELU(alpha=1.0, inplace=True)
    )
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, tra

In [6]:
net.decoder_blocks

ModuleList(
  (0): _UpSamplingBlock(
    (conv_block): _ConvBlock(
      (activation): ReLU(inplace=True)
      (block): Sequential(
        (0): Conv2d(768, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (up_sampling): Upsample(scale_factor=2.0, mode=bilinear)
  )
  (1): _UpSamplingBlock(
    (conv_block): _ConvBlock(
      (activation): ReLU(inplace=True)
      (block): Sequential(
        (0): Conv2d(384, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 

In [7]:
net.head

Conv2d(64, 3, kernel_size=(1, 1), stride=(1, 1))

In [8]:
yy = net(x)

In [19]:
y = (torch.randn(1, 256, 256) > 0).long()

In [9]:
yy.shape

torch.Size([1, 3, 256, 256])

In [21]:
y

tensor([[[1, 0, 1,  ..., 0, 1, 0],
         [1, 0, 1,  ..., 0, 1, 0],
         [1, 1, 0,  ..., 1, 1, 0],
         ...,
         [1, 0, 0,  ..., 0, 1, 1],
         [0, 0, 1,  ..., 1, 1, 0],
         [0, 0, 1,  ..., 0, 0, 0]]])

In [54]:
loss = nn.CrossEntropyLoss()

In [55]:
loss(yy, y)

tensor(1.2502, grad_fn=<NllLoss2DBackward>)

In [10]:
yy

tensor([[[[-0.1692,  0.1223,  0.1750,  ..., -0.1869, -0.0585,  0.0462],
          [-0.1302, -0.0230,  0.3185,  ..., -0.3760,  0.0204, -0.0686],
          [-0.1062, -0.0216,  0.4592,  ...,  0.0990,  0.0808, -0.1419],
          ...,
          [ 0.1386, -0.2856,  0.3074,  ..., -0.3874, -0.0322,  0.0503],
          [ 0.3562, -0.0960,  0.0815,  ...,  0.1893,  0.1438,  0.2804],
          [-0.2106, -0.1988,  0.0016,  ..., -0.0031, -0.2820,  0.0113]],

         [[-0.1542, -0.1322, -0.3917,  ..., -0.2297, -0.2328,  0.0103],
          [ 0.1040,  0.2189, -0.3661,  ...,  0.4818, -0.3737,  0.1117],
          [ 0.0735, -0.6487, -0.1899,  ...,  0.2213, -0.1529, -0.1020],
          ...,
          [-0.2046, -0.1477,  0.2941,  ...,  0.0652, -0.7276,  0.1676],
          [ 0.0413, -0.2013, -0.3192,  ..., -0.4947, -0.1179, -0.1000],
          [-0.4108,  0.0199,  0.2238,  ..., -0.4482, -0.2370,  0.0119]],

         [[ 0.0834,  0.1303,  0.0629,  ...,  0.4766, -0.0481,  0.2538],
          [ 0.1218,  0.1324,  

In [39]:
from torchsummary import summary

In [47]:

summary(net, (1, 256, 256), device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
├─ModuleList: 1                          []                        --
|    └─DownSamplingBlock: 2-1            [-1, 64, 128, 128]        --
|    |    └─ConvBlock: 3-1               [-1, 64, 256, 256]        37,824
|    |    └─MaxPool2d: 3-2               [-1, 64, 128, 128]        --
|    └─DownSamplingBlock: 2-2            [-1, 128, 64, 64]         --
|    |    └─ConvBlock: 3-3               [-1, 128, 128, 128]       221,952
|    |    └─MaxPool2d: 3-4               [-1, 128, 64, 64]         --
|    └─DownSamplingBlock: 2-3            [-1, 256, 32, 32]         --
|    |    └─ConvBlock: 3-5               [-1, 256, 64, 64]         886,272
|    |    └─MaxPool2d: 3-6               [-1, 256, 32, 32]         --
|    └─DownSamplingBlock: 2-4            [-1, 512, 32, 32]         --
|    |    └─ConvBlock: 3-7               [-1, 512, 32, 32]         3,542,016
├─ModuleList: 1                          []                     

Layer (type:depth-idx)                   Output Shape              Param #
├─ModuleList: 1                          []                        --
|    └─DownSamplingBlock: 2-1            [-1, 64, 128, 128]        --
|    |    └─ConvBlock: 3-1               [-1, 64, 256, 256]        37,824
|    |    └─MaxPool2d: 3-2               [-1, 64, 128, 128]        --
|    └─DownSamplingBlock: 2-2            [-1, 128, 64, 64]         --
|    |    └─ConvBlock: 3-3               [-1, 128, 128, 128]       221,952
|    |    └─MaxPool2d: 3-4               [-1, 128, 64, 64]         --
|    └─DownSamplingBlock: 2-3            [-1, 256, 32, 32]         --
|    |    └─ConvBlock: 3-5               [-1, 256, 64, 64]         886,272
|    |    └─MaxPool2d: 3-6               [-1, 256, 32, 32]         --
|    └─DownSamplingBlock: 2-4            [-1, 512, 32, 32]         --
|    |    └─ConvBlock: 3-7               [-1, 512, 32, 32]         3,542,016
├─ModuleList: 1                          []                     